# A Reservoir Computing Approach to Generalized Learning

## 1. Dependencies and Constants
### 1.1 Imports

In [1]:
import sys
parentdir = '../'
sys.path.insert(0, parentdir) 

import numpy as np
import pandas as pd # only used to read the MNIST data set
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

from pyESN import *
from SGD import SoftmaxSGD
from utils import *
from RLS import *


import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, \
        confusion_matrix, classification_report, ConfusionMatrixDisplay

In [4]:
plt.style.use('dark_background')
plt.rc('font', family='serif')
plt.rc('xtick', labelsize='xx-large')
plt.rc('ytick', labelsize='xx-large')
plt.rc('axes', labelsize='xx-large', titlesize='xx-large')
plt.rc('legend',fontsize=16)
%matplotlib inline

### 1.2 Dataset Loading


In [5]:
IMG_ROWS, IMG_COLS = 28,28    # number of rows and columns in mnist images
data_path = '../assets/dataset/'
df_train = pd.read_csv(os.path.join(data_path, 'mnist_train.csv'))
df_test = pd.read_csv(os.path.join(data_path, 'mnist_test.csv'))

In [6]:
X_train_ = df_train.drop(columns='label').to_numpy()
X_test_ = df_test.drop(columns='label').to_numpy()

y_train = df_train['label'].to_numpy()
y_test = df_test['label'].to_numpy()

X_train_, y_train = shuffle(X_train_, y_train, random_state=42)

Y_train = pd.get_dummies(y_train).to_numpy() # Convert categorical variable into dummy/indicator variables.
Y_test = pd.get_dummies(y_test).to_numpy()

IMG_LABEL = np.unique(y_test)
IMG_LABEL_ONE_HOT = np.eye(len(IMG_LABEL), dtype=np.float)     # array of one-hot label encodings

### 1.3 Preliminary

#### 1.3.1 [Function] Display the Stability of the Reservoir
Display the transient behavior of the reservoir by initializing the reservoir to random values and updating the reservoir over a number of time steps. Here we select 20 random neurons for display.


In [7]:
def display_reservoir_damping(states, N_neurons=20, iter_times=int(1e2)):
    '''
    In this function, M is the reservoir states matrix, whose dimension is (n_steps, N)
    And N is the dimension of reservoir weights matrix (i.e. the number of neurons ?). 
        (NB:Usually, the dimension of states matrix should be n_samples*N)
    Args:
        inputs (2d-array): matrix of all images, N_samples*K
        N_neurons (int): the number of neurons to display
    '''
    # check if the iteration times exceed the maximum threshold
    iter_times = states.shape[0] if iter_times > states.shape[0] else iter_times
    # if reservoir size is greater than N_neurons, select N_neurons random neurons for display
    cells = np.random.choice(np.arange(states.shape[1]), N_neurons)  if X_SIZE > 20 else np.arange(X_SIZE)     
    # format the plot area
    n_plots = min(states.shape[1], N_neurons)
    rows = int(np.floor(np.sqrt(n_plots))); cols = int(np.ceil(n_plots/rows))
    x_axis = np.arange(iter_times, dtype=np.int)   # set the x-axis as the iteration times (n_steps)
    # Display the transient behaviour of neurons
    fig, _axs = plt.subplots(nrows=rows, ncols=cols, figsize=(rows*4,cols*2.5))
    plt.suptitle(f'Transient Behaviour of randomly {N_neurons} neurons (y-axis:state, x-axis:epochs)')
    print('[INFO] Plotting Transient Behaviour ......')
    for p in tqdm(range(n_plots)):
        idx = cells[p] 
        row = p//cols; col = p % cols
        _axs[row][col].plot(x_axis, states[:iter_times, idx])
        _axs[row][col].set_title('Neurons: {:d}'.format(idx))
        _axs[row][col].set_xscale('log')
    fig.tight_layout()
    plt.show()


#### 1.3.2 [Function] Display Misclassified Images

In [8]:
import math
import random
def display_mis_images(mis_lbls):
    len_lbls = len(mis_lbls) if len(mis_lbls)<=25 else 25
    num = math.ceil(math.sqrt(len_lbls))
    
    fig = plt.figure(figsize=(num**2,num**2)) if num > 3 else plt.figure(figsize=((num+1)**2,(num+1)**2))
    for count, index in tqdm(enumerate(mis_lbls)):
        ax = fig.add_subplot(num, num, count + 1, xticks=[], yticks=[])
        image = X_test_[index[0]].reshape(28,28) 
        ax.set_title(f'Predict Label - True Label\n{mis_lbls[count][1:]}', fontsize=20)
        ax.imshow(image)

        if count==len_lbls-1:
            break

#### 1.3.3 [Function] Evaluation (Accuracy/precision/F1)


In [9]:
def _eval(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    evals = np.around(np.array([acc, prec, f1]), 4)
    
    print(classification_report(y_true, y_pred, target_names=IMG_LABEL.astype(str)))
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(cm, display_labels=IMG_LABEL.astype(str))
    disp.plot()
    
    print('_'*50, '\nAccuracy\tPrecison\tF1 score\n', '_'*50)
    print(f'{evals[0]}\t\t{evals[1]}\t\t{evals[2]}\n', '-'*50)
    return evals

### 1.4 Pre-Processing

#### 1.4.1 Normalization & Standardization

In [10]:
from sklearn.preprocessing import StandardScaler, Normalizer

def pre_process(X_train_, X_test_):
    transformer = Normalizer()
    X_train_ = transformer.fit_transform(X_train_)
    X_test_ = transformer.transform(X_test_)
    scaler = StandardScaler()
    X_train_ = scaler.fit_transform(X_train_)
    X_test_ = scaler.transform(X_test_)
    return X_train_, X_test_

#### 1.4.2 HOG

In [11]:
from skimage.feature import hog

def calc_hog_features(X, image_shape=(28, 28), cell=(8, 8), block=(2,2)):
    fd_list = []
    for row in tqdm(X):
        img = row.reshape(image_shape)
        fd = hog(img, orientations=8, pixels_per_cell=cell, cells_per_block=block)
        fd_list.append(fd)
    
    return np.array(fd_list)

#### Pre-Processing Flow

HOG $\Longrightarrow$ Normalization $\Longrightarrow$ Standardize

In [12]:
# HOG
is_HOG = True; cell_size = (3,3); block_size = (2,2); image_shape = (28,28)
X_train = calc_hog_features(X_train_, image_shape=image_shape, cell=cell_size, block=block_size) if is_HOG else X_train_ 
X_test = calc_hog_features(X_test_, image_shape=image_shape, cell=cell_size, block=block_size) if is_HOG else X_test_
print(X_train.shape, X_test.shape)

# Normalize &Standardize
X_train, X_test = pre_process(X_train, X_test)


## 2. ESN Setting

In [13]:
from sklearn.linear_model import SGDClassifier

U_SIZE = len(X_train[0])        # input vector is size of flattened image
O_SIZE = len(IMG_LABEL)  # image pair one-hot label size
rng = np.random.RandomState(42)  # wtf 42? You know, just Superstition & Metaphysics
clf = SGDClassifier(loss='modified_huber', penalty='l2', alpha=0.0001, max_iter=11, epsilon=0.1068,
                   class_weight='balanced', average=True)

In [14]:
X_SIZE = 400
esn = ESN(n_inputs = U_SIZE,
          n_outputs = O_SIZE,
          n_reservoir = X_SIZE,
          W_in_scaling = 0.6943,
          teacher_forcing = False,
          is_SLM=True, intensity=0.5418,
          leaky_rate = 0.8268,
          wash_out = 200,
          noise = 5e-3,
          sparsity = 0.03318,
          spectral_radius = 0.2829,
          learn_method='SGD', SGD_clf=clf,
          random_state = rng,
          silent = False)

In [15]:
pred_train = esn.fit(X_train, y_train) if esn.learn_method=='SGD' else esn.fit(X_train, Y_train)
pred_train = np.argmax(pred_train, axis=1)
eval_train = _eval(y_train, pred_train)

In [16]:
origin_states = esn.states[:, :X_SIZE]
display_reservoir_damping(origin_states, N_neurons=20, iter_times=int(5e4))

In [17]:
y_pred_test = esn.predict(X_test, continuation=False)
eval_test = _eval(y_test, y_pred_test)

mis_lbls = [(idx, y_pred_test[idx], y_test[idx]) \
            for idx, lbl in enumerate(y_pred_test) if lbl != y_test[idx]]
display_mis_images(mis_lbls)

## 3.Hyper Parameter Tuning 

In [18]:
X_SIZE = 50

### 3.1 Bayesian Optimization

#### (1) Define Black box function

In [19]:
from sklearn.model_selection import train_test_split

def esn_black_box(sparsity=0.2123, spectral_radius=0.1818, W_in_scaling=0.0205, leaky_rate=0.8324, 
                  intensity=0.9699, noise=1e-3, ridge_noise=1e-3, 
                  max_iter=11, epsilon=0.1, alpha=1e-4):
    """Function with unknown internals we wish to maximize"""
    
    clf = SGDClassifier(loss='modified_huber', penalty='l2', 
                        max_iter=max_iter, epsilon=epsilon, alpha = alpha,
                       class_weight='balanced', average=True)
    esn = ESN(n_inputs = U_SIZE,
        n_outputs = O_SIZE,
        n_reservoir = X_SIZE,
        sparsity = sparsity,
        spectral_radius = spectral_radius,
        W_in_scaling = W_in_scaling,
        leaky_rate = leaky_rate,
        teacher_forcing = False,
        is_SLM=True, intensity=intensity,
        learn_method='SGD', SGD_clf=clf,
        random_state = rng,
        silent = True)
    X_, X_val, y_, y_val = train_test_split(X_train, y_train, train_size=0.8)
    pred_train = esn.fit(X_, y_)
    pred_val = esn.predict(X_val, continuation=False)
    acc = accuracy_score(y_val, pred_val)
    return acc    

acc = esn_black_box()  # Example of black box esn function

#### (2) Define Bayesian Optimizer

In [20]:
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs


# Bounded region of parameter space, specify the minimum and maximum values 
# that can be probed for each parameter in order for it to work
opti_mode = 'INIT' # or 'INIT'

init_pbounds = { "spectral_radius": (1e-4, 1.5),
            "leaky_rate": (1e-4, 1),
            "sparsity": (1e-4, 1),
            "W_in_scaling": (1e-4, 2),
            "intensity": (1e-4, 2),
            "noise": (1e-5, 1e-2)}
init_params = {"spectral_radius": 0.3294, "sparsity": 0.2469,
                "leaky_rate": 0.8116, "W_in_scaling": 0.02683, 
                "intensity": 0.9714, "noise": 4e-3}

train_pbounds = { "max_iter": (2, 40),
                "epsilon": (0, 1),
                "alpha": (1e-5,1e-2)}
train_params = {"max_iter": 11, "epsilon": 0.1, "alpha": 1e-4}

pbounds = init_pbounds if opti_mode=='INIT' else train_pbounds
params = init_params if opti_mode=='INIT' else train_params

In [21]:
optimizer = BayesianOptimization(
    f=esn_black_box,
    pbounds=pbounds,
    verbose = 2,
    random_state=42,
)

# Saving progress
logger = JSONLogger(path="./logs.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

# Guide the optimization by specifying specific points to be probed
optimizer.probe(params = params, lazy=True)

#### (3) Start Bayesian Optimization 

There are many parameters you can pass to maximize, nonetheless, the most important ones are:

- `n_iter`: How many steps of bayesian optimization you want to perform. The more steps the more likely to find a good maximum you are.
- `init_points`: How many steps of random exploration you want to perform. Random exploration can help by diversifying the exploration space.

In [22]:
optimizer.maximize(
    init_points=3,
    n_iter=6,
)

for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

In [23]:
new_optimizer = BayesianOptimization(
    f=esn_black_box,
    pbounds=pbounds,
    verbose = 2,
    random_state=42,
)

load_logs(new_optimizer, logs=["./logs.json"]);

print("New optimizer is now aware of {} points.".format(len(new_optimizer.space)))
new_optimizer.maximize(
    init_points=0,
    n_iter=20,
)

 ### 3.2 Grid Search CV

In [24]:
from sklearn.model_selection import GridSearchCV

In [25]:
esn_cv = ESN(n_inputs = U_SIZE,
            n_reservoir = 10,
            n_outputs = O_SIZE,
            teacher_forcing = False,
            is_SLM = True,
            learn_method='SGD', SGD_clf=clf,
            random_state = rng,
            silent = True)

param_grid = {
    "spectral_radius": np.arange(0.5,1,0.2),
    "sparsity": np.arange(0.5,1,0.2),
    "leaky_rate": np.arange(0.1, 1.0, 0.1),
    "W_in_scaling": np.arange(0.1, 0.9, 0.1),
}

esn_GS = GridSearchCV(esn_cv, param_grid, 
                    cv=3, scoring='accuracy')

In [ ]:
esn_GS.fit(X_train, y_train)
esn_GS.best_estimator_

In [ ]:
pred_test = esn_GS.best_estimator_.predict(X_test, continuation=False)
eval_test = _eval(y_test, pred_test)

### 3.3 Randomized Search CV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, loguniform

esn_cv = ESN(n_inputs = U_SIZE,
            n_reservoir = 10,
            n_outputs = O_SIZE,
            teacher_forcing = False,
            is_SLM = True,
            learn_method='SGD', SGD_clf=clf,
            random_state = rng,
            silent = True)

param_grid = {
    "spectral_radius": uniform(loc=0.3, scale=0.7),
    "sparsity": uniform(loc=0.3, scale=0.7),
    "leaky_rate": uniform(loc=0.1, scale=0.7),
    "W_in_scaling": loguniform(a=0.1, b=1, loc=0, scale=1),
    "intensity": loguniform(a=0.01, b=1, loc=0, scale=1),
}

acc_score = make_scorer(custom_acc_score, greater_is_better=True, needs_proba=True)
esn_RS = RandomizedSearchCV(esn_cv, param_grid, random_state=42, 
                             n_iter=5, scoring=acc_score)

In [ ]:
esn_RS.fit(X_train, y_train)

best_esn_RS = esn_RS.best_estimator_
y_pred_test = best_esn_RS.predict(X_test, continuation=False)
eval_test = _eval(y_test, y_pred_test)

### [Training] Adam + Nesterov

#### Initialze States

In [ ]:
from sklearn.neural_network import MLPClassifier

## initialize training data 
X = esn.states; y = y_train

## initialize test data
X_test_2d = X_test.reshape(X_test.shape[0], -1)
n_test = X_test_2d.shape[0]

init_input = np.zeros(esn.n_inputs); init_state = np.zeros(esn.n_reservoir); init_output = np.zeros(esn.n_outputs)
inputs_test = np.vstack([init_input, X_test_2d])
states_test = np.vstack([init_state, np.zeros((n_test, esn.n_reservoir))])
outputs_test = np.vstack([init_output, np.zeros((n_test, esn.n_outputs))])

for n in tqdm(range(n_test)):
    states_test[n+1, :] = esn._update(states_test[n, :], inputs_test[n+1, :], outputs_test[n, :])

states_test_extended = np.hstack((states_test[1:, :], X_test_2d))

#### Train and Evaluation

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(800,), activation='tanh', 
                    solver='adam', alpha=3e-3, batch_size='auto',
                    learning_rate='adaptive', learning_rate_init=5e-4,
                    max_iter=800, random_state=42,
                    momentum=0.9, nesterovs_momentum=True,
                   beta_1=0.9, beta_2=0.999, epsilon=1e-9)
clf.fit(X, y)

In [ ]:
pred_proba_train = clf.predict_proba(X)
pred_train = np.argmax(pred_proba_train, axis=1)
eval_test = _eval(y_train, pred_train)

pred_proba_test = clf.predict_proba(states_test_extended)
pred_test = np.argmax(pred_proba_test, axis=1)
eval_test = _eval(y_test, pred_test)